In [1]:
# Displaying graphs
import plotly.io as pio
pio.renderers.default = 'iframe'

# Dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Data
import pandas as pd
import numpy as np
import cufflinks as cf
import calendar
from datetime import datetime
from dateutil.relativedelta import *
import re

# Charts
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

### Helper functions

In [2]:
# Needed for easy separation
def globalization(strok:str):
    return strok.replace(';', ', ')

# It is necessary to calculate how many percent the indicator has increased or decreased
def num_str(n1, n2):
    if n1 > n2:
        perc = (n1-n2)/n1*100
        symb = '▼'
    elif n1 < n2:
        perc = (n2-n1)/n1*100
        symb = '▲'
    
    perc_symb = symb + ' ' + str(round(perc,2)) + '%'
    return [str(n1), str(n2), perc_symb]

### Expanding the dataframe

In [3]:
# Import dataframe
united_df = pd.read_csv('Steam Store Games (Clean dataset)/steam.csv')

# rating_difference is needed to determine popular games that users liked or did not like
united_df['rating_difference'] = united_df['positive_ratings'] - united_df['negative_ratings']

# rating_sum is needed to determine the total number of reviews
united_df['rating_sum'] = united_df['positive_ratings'] + united_df['negative_ratings']

# Changing the column for better visual display
united_df['platforms'] = united_df['platforms'].apply(globalization)

# Initially, the column is not considered a date column
united_df['release_date'] = pd.to_datetime(united_df['release_date'])

# Adding the columns for better visual display
united_df['free_to_play'] = (united_df['price'] == 0).apply(lambda i : 'Yes' if i == True else 'No')
united_df['english_str'] = united_df['english'].apply(lambda i : 'Yes' if i == 1 else 'No')

# Adding the columns for simplify plotting
united_df['p'] = united_df['rating_difference'].apply(lambda x: 1 if x > 0 else 0)
united_df['n'] = united_df['rating_difference'].apply(lambda x: 1 if x < 0 else 0)
united_df['s'] = united_df['rating_difference'].apply(lambda x: 1 if x == 0 else 0)
united_df['ones'] = 1

### Calculating values for data in blocks

In [4]:
# Dares for separation previous and present 6 months
date_1 = datetime.fromisoformat('2018-11-01')
date_2 = datetime.fromisoformat('2019-05-01')
date_3 = datetime.fromisoformat('2018-05-01')

# Creation of dataframes for previous and present 6 months
present_df = united_df[(united_df['release_date'] >= date_1) & (united_df['release_date'] < date_2)]
previous_df = united_df[(united_df['release_date'] >= date_3) & (united_df['release_date'] < date_1)]

# Calculating the number of new games
num_present_1 = len(present_df)
num_previous_1 = len(previous_df)

mas_1 = num_str(num_previous_1, num_present_1)

# Calculating the number of new games with english language support
num_present_2 = present_df['english'].sum()
num_previous_2 = previous_df['english'].sum()

mas_2 = num_str(num_previous_2, num_present_2)

# Calculating the average game price
num_present_3 = present_df['price'].sum()/num_present_1
num_previous_3 = previous_df['price'].sum()/num_previous_1

mas_3 = num_str(round(num_previous_3, 2), round(num_present_3, 2))

# Calculating the average playtime
num_present_4 = present_df['average_playtime'].sum()/num_present_1
num_previous_4 = previous_df['average_playtime'].sum()/num_previous_1

mas_4 = num_str(round(num_previous_4, 2), round(num_present_4, 2))

### Plots for dashboards

In [5]:
group_sum_1 = united_df.groupby('publisher')[['p', 'n', 's', 'ones']].sum()
sort_group_1 = group_sum_1.sort_values(by='ones').tail(n=10)

bar_1 = px.bar(sort_group_1, 
               x=['p', 's', 'n'],
               orientation="h",
               labels={'publisher':'Publisher', 'value':'Number of games'},
               title='Top 10 publishers by number of games',
                color_discrete_map={'n': '#FF6666', 's': '#C0C0C0', 'p':'#66B2FF'})

bar_1.update_layout(showlegend=False)
bar_1.update_traces(opacity=0.6)

In [6]:
group_sum_2 = united_df.groupby('developer')[['p', 'n', 's', 'ones']].sum()
sort_group_2 = group_sum_2.sort_values(by='ones').tail(n=10)

bar_2 = px.bar(sort_group_2, 
               x=['p', 's', 'n'],
               orientation="h",
               labels={'developer':'Developer', 'value':'Number of games'},
               title='Top 10 developers by number of games',
               color_discrete_map={'n': '#FF6666', 's': '#C0C0C0', 'p':'#66B2FF'})

bar_2.update_layout(showlegend=False)
bar_2.update_traces(opacity=0.6)

In [7]:
group_sum_3 = united_df.groupby('developer')['rating_difference'].sum()
sort_group_3 = group_sum_3.sort_values()

bar_3 = px.bar(sort_group_3.tail(n=10), 
               orientation="h",
               labels={'developer':'Developer', 'value':'Difference in ratings'},
               title='Top 10 developers by difference in ratings')

bar_3.update_layout(showlegend=False)
bar_3.update_traces(marker_color='#66B2FF', opacity=0.6)

In [8]:
bar_4 = px.bar(sort_group_3.head(10), 
               orientation="h",
               labels={'developer':'Developer', 'value':'Difference in ratings'},
               title='Worst 10 developers by difference in ratings')

bar_4.update_layout(showlegend=False)
bar_4.update_traces(marker_color='#FF6666', opacity=0.6)

In [9]:
group_sum_5 = united_df.groupby('publisher')['rating_difference'].sum()
sort_group_5 = group_sum_5.sort_values()

bar_5 = px.bar(sort_group_5.tail(n=10), 
               orientation="h",
               labels={'developer':'Developer', 'value':'Difference in ratings'},
               title='Top 10 developers by difference in ratings')
               
bar_5.update_layout(showlegend=False)
bar_5.update_traces(marker_color='#66B2FF', opacity=0.6)

In [10]:
bar_6 = px.bar(sort_group_5.head(10), 
               orientation="h",
               labels={'developer':'Developer', 'value':'Difference in ratings'},
               title='Worst 10 developers by difference in ratings')

bar_6.update_layout(showlegend=False)
bar_6.update_traces(marker_color='#FF6666', opacity=0.6)

In [11]:
bar_7 = px.bar(united_df.sort_values(by='rating_sum').tail(n=10),
               x=['positive_ratings','negative_ratings'],
               y='name',
               orientation="h",
               labels={'name':'Game', 'value':'Number of ratings'},
               title='Top 10 games by number of ratings',
               color_discrete_map={'positive_ratings': '#66B2FF', 'negative_ratings': '#FF6666'})

bar_7.update_layout(legend_title_text='Rating type')
bar_7.update_traces(opacity=0.6)

In [12]:
month_year_df = united_df.sort_values(by='release_date')
month_year_df['month_year'] = month_year_df['release_date'].dt.month_name() + ' ' + month_year_df['release_date'].dt.year.apply(str)
month_year_group = month_year_df.groupby(by=['month_year'], sort=False)[['p', 'n', 's']].sum()

bar14 = go.Figure(data=[go.Bar(x=month_year_group.index, y=list(month_year_group['p']), 
                               marker_color='#66B2FF', name='Positive'),
                        go.Bar(x=month_year_group.index, y=list(month_year_group['s']), 
                               marker_color='#C0C0C0', name='Same'),
                        go.Bar(x=month_year_group.index, y=list(month_year_group['n']),
                               marker_color='#FF6666', name='Negative')])

bar14.update_layout(barmode='stack',
                    legend_title_text='Rating type',
                    title={'text':'Number of games released each month'},
                    yaxis_title='Number of games',
                    xaxis=dict(rangeslider=dict(visible=True)))

### Styles

In [13]:
# styles of the top elements
for_game_smile = {'fontSize':'48px',
                  'margin':'30px auto 0',
                  'textAlign':'center',}

for_header_title = {'color':'#FFFFFF',
                    'fontSize':'48px',
                    'fontWeight':'bold',
                    'textAlign':'center',
                    'margin':'0 auto',}

for_header_description = {'color':'#FFFFFF',
                          'margin':'4px auto 50px',
                          'textAlign':'center',
                          'maxWidth':'450px',}

for_header = {'backgroundColor':'#212121',
              'display':'flex',
              'flexDirection':'column',
              'justifyContent':'center',}

for_bar14 = {'width':'96%',
             'textAlign':'center',
             'boxShadow': '#a6a6a6 0px 5px 15px',
             'margin':'15px auto 10px',}

# background styles
main_div_style = {'backgroundColor':'#FFFFFF', 
                  'padding':'0', 
                  'width':'100%', 
                  'height':'100%',
                  'display':'flex',
                  'position':'fixed',
                  'flexDirection':'column',
                  'top':'0',
                  'left':'0',
                  'bottom':'0','overflow':'auto',}

# data style
for_stat_text = {'backgroundColor':'#FFFFFF',
                 'width':'300px',
                 'border-radius':'5px',
                 'boxShadow': '#a6a6a6 0px 5px 15px',
                 'padding':'10px',
                 'position':'relative',
                 'margin':'15px auto 0',}

for_head = {'color':'#212121',
            'fontSize':'22px',
            'fontWeight':'bold',
            'textAlign':'start',
            'fontFamily': 'Arial',
            'margin':'0 auto',}

for_descr_present = {'color':'#757575',
                     'fontSize':'16px',
                     'fontFamily': 'Arial',
                     'textAlign':'start',
                     'margin':'0 auto',}

for_present_green = {'color':'#009900',
                     'fontSize':'20px',
                     'fontFamily': 'Arial',
                     'textAlign':'start',
                     'margin':'0 auto',}

for_present_red = {'color':'#990000',
                   'fontSize':'20px',
                   'fontFamily': 'Arial',
                   'textAlign':'start',
                   'margin':'0 auto',}

for_descr_previous = {'color':'#757575',
                      'fontSize':'16px',
                      'fontFamily': 'Arial',
                      'textAlign':'start',
                      'margin':'0 auto',}

# if the indicator has increased
for_previous_green = {'color':'#009900',
                      'fontSize':'18px',
                      'textAlign':'center',
                      'fontFamily': 'Arial',
                      'textAlign':'start',
                      'margin':'0 auto',}

# if the indicator has decreased
for_previous_red = {'color':'#990000',
                    'fontSize':'18px',
                    'textAlign':'center',
                    'fontFamily': 'Arial',
                    'textAlign':'start',
                    'margin':'0 auto',}

for_percent_green = {'color':'#009900',
                     'fontSize':'24px',
                     'fontFamily': 'Arial',
                     'position': 'absolute',
                     'top':'45px',
                     'right':'10px',
                     'margin':'0 auto',}

for_percent_red = {'color':'#990000',
                   'fontSize':'24px',
                   'fontFamily': 'Arial',
                   'position': 'absolute',
                   'top':'45px',
                   'right':'10px',
                   'margin':'0 auto',}

for_div_texts = {'display':'flex',
                 'flexDirection':'row',
                 'justifyContent':'space-evenly',}

# common block style 
for_pie_bar = {'display':'flex',
                 'flexDirection':'row',
                 'justifyContent':'space-evenly',
                 'width':'96%',
                 'margin':'15px auto 0',}

# pies div
for_pie_div = {'backgroundColor':'#FFFFFF',
               'width':'37%',
               'boxShadow': '#a6a6a6 0px 5px 15px',
               'margin':'0px auto',}

# bars div
for_bar_div = {'backgroundColor':'#FFFFFF',
               'width':'62%',
               'boxShadow': '#a6a6a6 0px 5px 15px',
               'margin':'0px auto',}

### App setup

In [16]:
app = dash.Dash(__name__)
server = app.server 
app.title = "Steam Store Games" 

text_descr_1 = 'A small dashboard that shows statistics from the kaggle dataset'
text_descr_2 = ': https://www.kaggle.com/nikdavis/steam-store-games'

# div blocks
steam_text = html.Div(children=[html.P(children='🎮', className='game_smile', style=for_game_smile ), 
                                html.H1(children='Steam Store Games (until april 2019)', 
                                        className='header_title', style=for_header_title),
                                html.P(children=text_descr_1 + text_descr_2, 
                                       className='header_description', style=for_header_description),],
                      className='header', style=for_header)

bar14_l = html.Div(children=dcc.Graph(figure=bar14), style=for_bar14)

stat_text_1 = html.Div(children=[html.P(children='New games',
                                      className='head_1', style=for_head),
                                 
                                 html.P(children='In last 6 monts',
                                        className='descr_present_1', style=for_descr_present),
                                 
                                 html.P(children=mas_1[1], 
                                        className='present_1', style=for_present_red),
                                 
                                 html.P(children='In previous 6 months',
                                        className='descr_previous_1', style=for_descr_previous),
                                 
                                 html.P(children=mas_1[0], 
                                        className='previous_1', style=for_previous_red),
                                 
                                 html.P(children=mas_1[2], 
                                        className='percent_1', style=for_percent_red)],
                       className = 'stat_text_1',
                       style=for_stat_text)

stat_text_2 = html.Div(children=[html.P(children='En language support',
                                      className='head_2', style=for_head),
                                 
                                 html.P(children='In last 6 monts',
                                        className='descr_present_2', style=for_descr_present),
                                 
                                 html.P(children=mas_2[1], 
                                        className='present_2', style=for_present_red),
                                 
                                 html.P(children='In previous 6 months',
                                        className='descr_previous_2', style=for_descr_previous),
                                 
                                 html.P(children=mas_2[0], 
                                        className='previous_2', style=for_previous_red),
                                 
                                 html.P(children=mas_2[2], 
                                        className='percent_2', style=for_percent_red)],
                       className = 'stat_text_2',
                       style=for_stat_text)

stat_text_3 = html.Div(children=[html.P(children='Average price',
                                      className='head_3', style=for_head),
                                 
                                 html.P(children='In last 6 monts',
                                        className='descr_present_3', style=for_descr_present),
                                 
                                 html.P(children=mas_3[1] + ' GBP', 
                                        className='present_3', style=for_present_green),
                                 
                                 html.P(children='In previous 6 months',
                                        className='descr_previous_3', style=for_descr_previous),
                                 
                                 html.P(children=mas_3[0] + ' GBP', 
                                        className='previous_3', style=for_previous_green),
                                 
                                 html.P(children=mas_3[2], 
                                        className='percent_3', style=for_percent_green)],
                       className = 'stat_text_3',
                       style=for_stat_text)

stat_text_4 = html.Div(children=[html.P(children='Average playtime',
                                      className='head_4', style=for_head),
                                 
                                 html.P(children='In last 6 monts',
                                        className='descr_present_4', style=for_descr_present),
                                 
                                 html.P(children=mas_4[1] + ' hours', 
                                        className='present_4', style=for_present_red),
                                 
                                 html.P(children='In previous 6 months',
                                        className='descr_previous_4', style=for_descr_previous),
                                 
                                 html.P(children=mas_4[0] + ' hours', 
                                        className='previous_4', style=for_previous_red),
                                 
                                 html.P(children=mas_4[2], 
                                        className='percent_4', style=for_percent_red)],
                       className = 'stat_text_4',
                       style=for_stat_text)

bars_drop_dict = {'english_str':'English language support',
                  'required_age':'Required age',
                  'platforms':'Platforms',
                  'free_to_play':'Free to play'}

pie_div = html.Div(children=[dcc.Dropdown(id='dropdown-pie',
                                           options=[{'label':'English language support',
                                                     'value':'english_str'},
                                                    {'label':'Required age',
                                                     'value':'required_age'},
                                                    {'label':'Platforms',
                                                     'value':'platforms'},
                                                    {'label':'Free to play',
                                                     'value':'free_to_play'}],
                                           value='english_str'),
                              dcc.Graph(id='dropdown-pie-output')],
                   className='pie_div', 
                   style=for_pie_div)

bars_dict = {'bar_1':bar_1, 'bar_2':bar_2, 'bar_3':bar_3, 'bar_4':bar_4, 
             'bar_5':bar_5, 'bar_6':bar_6, 'bar_7':bar_7}

bar_div = html.Div(children=[dcc.Dropdown(id='dropdown-bar',
                                          options=[{'label':'Top 10 publishers by number of games',
                                                    'value':'bar_1'},
                                                   {'label':'Top 10 developers by number of games',
                                                    'value':'bar_2'},
                                                   {'label':'Top 10 developers by difference in ratings',
                                                    'value':'bar_3'},
                                                   {'label':'Worst 10 developers by difference in ratings',
                                                    'value':'bar_4'},
                                                   {'label':'Top 10 developers by difference in ratings',
                                                    'value':'bar_5'},
                                                   {'label':'Worst 10 developers by difference in ratings',
                                                    'value':'bar_6'},
                                                   {'label':'Top 10 games by number of ratings',
                                                    'value':'bar_7'}],
                                          value='bar_7'),
                             dcc.Graph(id='dropdown-bar-output')],
                   className='bar_div', 
                   style=for_bar_div)

div_texts = html.Div(children=[stat_text_1, stat_text_2, stat_text_3, stat_text_4], 
                     className='for_div_texts', style=for_div_texts)

pie_bar = html.Div(children=[pie_div, bar_div], 
                   className='for_pie_bar', style=for_pie_bar)

app.layout = html.Div(id = "main_div", children=[steam_text, div_texts, pie_bar, bar14_l], 
                      style = main_div_style)

# pie dropdown
@app.callback(Output('dropdown-pie-output', 'figure'),
              [Input('dropdown-pie', 'value')])
def update_pie(value):
    pie1 = px.pie(united_df, names=value, title=bars_drop_dict[value])
    pie1.update_traces(opacity=0.6)
    return pie1

# bar dropdown
@app.callback(Output('dropdown-bar-output', 'figure'),
              [Input('dropdown-bar', 'value')])
def update_pie(value):
    return bars_dict[value]


if __name__ == "__main__":
    app.run_server(port = 8080)